In [49]:
import dlib
import cv2
import numpy as np
import tensorflow as tf
from keras.preprocessing import image

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import models, layers, activations
from tensorflow.keras.layers import Conv2D, TimeDistributed, BatchNormalization, MaxPooling2D, Flatten, Bidirectional, Dense,Dropout,Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.layers import Input

In [50]:
def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((20, 2), dtype=dtype)
    # for only lip landmarks
    n = 0
    for i in range(48, shape.num_parts):
        coords[n] = (shape.part(i).x, shape.part(i).y)
        n += 1

    # return the list of (x, y)-coordinates
    return coords

In [51]:
def get_rect(shape):
    rw = 0
    rh = 0
    rx = 65535
    ry = 65535
    for (x,y) in shape:
        rw = max(rw,x) # w
        rh = max(rh,y) # h
        rx = min(rx,x) # x
        ry = min(ry,y) # y
    return (rx,ry,rw-rx,rh-ry)

In [63]:
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("data/shape_predictor_68_face_landmarks.dat") # predict 해주는애       

count = 0

cap = cv2.VideoCapture(0)
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        image = cv2.resize(frame, dsize = (640, 480), interpolation = cv2.INTER_AREA)
        img = image.copy()
        img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        face_detector = detector(img_gray, 1)
        cv2.imshow('frame',image)  
        for face in face_detector:
            landmarks = predictor(image, face)
            landmarks = shape_to_np(landmarks)
            # select center of mouth
            x_list = [x[0] for x in landmarks]
            y_list = [y[1] for y in landmarks]
            x = sum(x_list)//20
            y = sum(y_list)//20
            dst = img[y-50:y+50, x-100:x+100].copy()
            img_tensor = img_to_array(dst) #tensor로 type 변경
            img_tensor /= 255. # scaling  # (100, 200, 3)
            img_tensor = np.expand_dims(img_tensor, axis = 0) #(1, 100, 200, 3)  
            # frame 별 묶기
            if count == 0:
                x_data = img_tensor
                count += 1
            else:
                x_data = np.concatenate((x_data, img_tensor), axis = 0)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()

In [64]:
# input x의 값
x_data[:1].shape

(1, 100, 200, 3)

In [65]:
data = x_data[:1].reshape(-1,1,100, 200, 3)

In [66]:
print(data.shape)

(1, 1, 100, 200, 3)


In [67]:
class MobileNet_LSTM(keras.Model):
    def __init__(self, num_classes):
        super(MobileNet_LSTM, self).__init__()
        
        self.mobile = TimeDistributed(MobileNet(include_top=False))

        #self.mobile = TimeDistributed(MobileNet(#weights='imagenet', include_top=False))

        self.bilstm = layers.Bidirectional(layers.LSTM(256), merge_mode='concat')
        self.dense = layers.Dense(num_classes, activation='softmax')
        self.max = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))
        self.dropout = layers.Dropout(0.2)
        self.flat= TimeDistributed(Flatten())

    def call(self, x):
        x = self.mobile(x)
        x = self.max(x)
        x = self.dropout(x)      
        x = self.flat(x)
        x = self.dropout(x)
        x = self.bilstm(x)
        return self.dense(x)

In [68]:
mobilenet = MobileNet_LSTM(num_classes = 3)

In [69]:
mobilenet.load_weights('MobileNet_weights')

In [70]:
answer = mobilenet.predict(data)
print(answer)

1/1 [==============================] - 2s 2s/step
[[0.29890463 0.43694827 0.26414707]]


In [71]:
words = ['우', '린', '니', '까']

In [72]:
sent = words[np.argmax(answer)]
print(sent)

린


In [73]:
string = ''
total = string + sent
print(total)

린
